### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [6]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [7]:
%sh aql -c "truncate test" -h "3.235.121.162"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [8]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [26]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpReadFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.exp.ListExp;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

import java.util.Random;

AerospikeClient client = new AerospikeClient("3.235.121.162", 3000);
Key key = new Key("test", "testset", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin b0 = new Bin("b00", Value.get(1)); //int bin

List<Value> myList = new ArrayList<Value>();
Bin listData = new Bin("data", Value.get(myList));
//Note: Adding empty bin "data" so that list size expression can execute to start with.

client.put(wPolicy, key, b0, listData);
System.out.println("Record: "+ client.get(null, key)); //check record got inserted

//Using expressions to read record metadata in PI that has size of record on device
Expression recSizeExp = Exp.build(Exp.deviceSize());
Record record = client.operate( wPolicy, key,   
          ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT) 
         );
System.out.println("Record Size in bytes: " + record.getValue("size"));
int iMaxListSize = 8500;
wPolicy.filterExp = Exp.build(Exp.lt(ListExp.size(Exp.listBin("data")),Exp.val(iMaxListSize)));
wPolicy.failOnFilteredOut = true;

try {
    int j=0;
    int k=0;
    Random rndj = new Random();
    Random rndk = new Random();
    
    while(true) {
        List<Value> values = new ArrayList<>();
        
        for (int i = 0; i < 500; i++) {
            //values.add(Value.get("111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111"));
            j=rndj.nextInt(1000000);
            k=rndk.nextInt(j);
            String strval = "aquic"+k+"kbrown"+j+"fox"+k+"jum"+j+"psri"+j+"ght"+j+"ove"+k+"rt"+j+"hela"+k+"zyd"+j+"og"+j+"aqu"+j+"ick"+k+"br"+j+"o"+k+"wnfo"+j+"xjum"+j+"ps"+k+"rig"+j+"hto"+k+"ver"+j+"the"+k+"la"+j+"zydog"+(j*5)+"aqu"+k+"ick"+j+"bro"+k+"w"+j+"nfoxjum"+j+"psrigh"+j+"tove"+j+"rthe"+i+"laz"+i+"ydog"+(j+10)+"end";
              values.add(Value.get(strval));
         }
        Expression recSizeExp = Exp.build(Exp.deviceSize());
        
        Record r1 = client.operate(wPolicy, key, 
                    ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                    ListOperation.appendItems("data", values, null));
        
        System.out.println("Record Size in bytes: text " + r1.getValue("size"));
        
        Record record = client.operate( null, key, 
                        ExpOperation.read("size", recSizeExp, ExpReadFlags.DEFAULT), 
                        ListOperation.size("data", null)  //<-- gives number of elements in list
                        );       
        
        System.out.println("Record Size in bytes: text " + record.getValue("size"));        
        System.out.println("Bin Array (each element = #128 bytes) Size: " + record.getValue("data"));
        
        
    }
} catch (AerospikeException e){
  int rc = e.getResultCode();
  if(rc == ResultCode.FILTERED_OUT){
    System.out.println("Array Limit Exceeded. ==> Split the Record.");
  }
  else{
    System.out.println("Operate error. Resultcode: "+ ResultCode.getResultString(rc));
  }
}


Record: (gen:288),(exp:417763034),(bins:(b00:1),(data:[]))
Record Size in bytes: 80
Record Size in bytes: text 80
Record Size in bytes: text 56192
Bin Array (each element = #128 bytes) Size: 500
Record Size in bytes: text 56192
Record Size in bytes: text 111808
Bin Array (each element = #128 bytes) Size: 1000
Record Size in bytes: text 111808
Record Size in bytes: text 167392
Bin Array (each element = #128 bytes) Size: 1500
Record Size in bytes: text 167392
Record Size in bytes: text 223040
Bin Array (each element = #128 bytes) Size: 2000
Record Size in bytes: text 223040
Record Size in bytes: text 278608
Bin Array (each element = #128 bytes) Size: 2500
Record Size in bytes: text 278608
Record Size in bytes: text 334112
Bin Array (each element = #128 bytes) Size: 3000
Operate error. Resultcode: Record too big


### Not a Bug
Just insert this one record. However start by shutting down the node, wiping the node data file and restart before running the test.  This will reset the moving average device compression ratio. Otherwise prior test updates may give erroneous compression_ratio number.

Use asadm to see device compression ratio.  
The original string with all 1's was extremely compressible. The new string created above with random numbers is down to 36% compressible. (Device compression ratio = compressed_size/original_size = 36%.)

```
Admin+> asinfo -v 'namespace/test' like compression_ratio
ec2-44-200-159-121.compute-1.amazonaws.com:3000 (44.200.159.121) returned:
device_compression_ratio=0.357; ...
```

and 334,112 bytes was the record size before we hit _Record Too Big_. 

```
334112/.357 = ~ 935,887 ....next append made it went over > 1MB.
```